In [4]:
using JuMP
using GLPK

# Crear el modelo
model = Model(GLPK.Optimizer)

# Parámetros
demanda = [180, 250, 190, 140, 220, 250] # Demanda de ventanas por mes
costos_produccion = [50, 45, 55, 52, 48, 50] # Costo de producción por ventana por mes
costos_almacenaje = [8, 10, 10, 10, 8, 8] # Costo de almacenaje por ventana por mes
capacidad = 225 # Capacidad máxima de producción por mes
n_meses = length(demanda)

# Variables de decisión
@variable(model, x[1:n_meses] >= 0, Int)  # Cantidad de ventanas producidas en cada mes
@variable(model, i[1:n_meses] >= 0, Int)  # Inventario al final de cada mes

# Función objetivo (minimizar los costos totales de producción y almacenaje)
@objective(model, Min, sum(costos_produccion[t] * x[t] for t in 1:n_meses) + 
                        sum(costos_almacenaje[t] * i[t] for t in 1:n_meses))

# Restricciones de demanda mensual
@constraint(model, i[1] == x[1] - demanda[1])  # Inventario al final del primer mes
for t in 2:n_meses
    @constraint(model, i[t] == i[t-1] + x[t] - demanda[t])  # Inventario para los siguientes meses
end

# Restricciones de capacidad de producción por mes
for t in 1:n_meses
    @constraint(model, x[t] <= capacidad)  # No se puede producir más de 225 ventanas al mes
end


In [5]:
# Resolver el problema
optimize!(model)

# Mostrar los resultados
println("Programa de producción óptimo por mes:")
for t in 1:n_meses
    println("Mes $t: Producción = ", value(x[t]), " ventanas, Inventario final = ", value(i[t]), " ventanas")
end

println("Costo total: ", objective_value(model))

Programa de producción óptimo por mes:
Mes 1: Producción = 205.0 ventanas, Inventario final = 25.0 ventanas
Mes 2: Producción = 225.0 ventanas, Inventario final = 0.0 ventanas
Mes 3: Producción = 190.0 ventanas, Inventario final = 0.0 ventanas
Mes 4: Producción = 160.0 ventanas, Inventario final = 20.0 ventanas
Mes 5: Producción = 225.0 ventanas, Inventario final = 25.0 ventanas
Mes 6: Producción = 225.0 ventanas, Inventario final = 0.0 ventanas
Costo total: 61795.0
